#### 1. Reward the top performer (student) of each school based on cumulative marks scored in last three years for all the subjects


In [4]:
import pandas as pd

# Load the data
file_2019 = "Bangalore Schools 2019.xlsx"
file_2020 = "Bangalore Schools 2020.xlsx"
file_2021 = "Bangalore Schools 2021.xlsx"

df_2019 = pd.read_excel(file_2019)
df_2020 = pd.read_excel(file_2020)
df_2021 = pd.read_excel(file_2021)

# Add a Year column to each dataset
df_2019['Year'] = 2019
df_2020['Year'] = 2020
df_2021['Year'] = 2021

# Combine all datasets
combined_df = pd.concat([df_2019, df_2020, df_2021], ignore_index=True)

# List of subject columns (excluding non-score columns)
subject_columns = ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry',
                   'Biology', 'History', 'Geography', 'Civics', 'Computer Science', 'Physical Education']

# Calculate total marks per student
combined_df['Total_Marks'] = combined_df[subject_columns].sum(axis=1)

# Calculate cumulative marks across 3 years
cumulative_marks = combined_df.groupby(['School', 'Student Roll', 'Student Name'])['Total_Marks'].sum().reset_index()

# Identify top performer in each school
top_performers = cumulative_marks.loc[cumulative_marks.groupby('School')['Total_Marks'].idxmax()]

# Display results
print(top_performers[['School', 'Student Name', 'Total_Marks']])


          School  Student Name  Total_Marks
19  Vidya Mandir  Nisha Saxena         2320


#### 2.Rank each student within their own school based on their total marks scored in the year 2020 and compare the marks of Rank 10 for each school by arranging them in descending order

In [31]:
# List of subject columns
subject_columns = ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry',
                   'Biology', 'History', 'Geography', 'Civics', 'Computer Science', 'Physical Education']

# Calculate total marks per student
df_2020["Total_Marks"] = df_2020[subject_columns].sum(axis=1)

# Rank students within each school based on total marks (higher marks = better rank)
df_2020["Rank"] = df_2020.groupby("School")["Total_Marks"].rank(method="dense", ascending=False)

# Filter to get only Rank 10 students from each school
rank_10_students = df_2020[df_2020["Rank"] == 10]

# Sort schools by Rank 10 student's marks in descending order
rank_10_sorted = rank_10_students.sort_values(by="Total_Marks", ascending=False)

# Select relevant columns for output
rank_10_sorted = rank_10_sorted[["School", "Student Name", "Total_Marks", "Rank"]]

# Display the sorted result
rank_10_sorted


,School,Student Name,Total_Marks,Rank
11,Vidya Mandir,Ganesh Sekhar,705,10.0
76,Birla HS,Derek Pinto,673,10.0
87,International,Atin Choudhari,647,10.0
28,St. Joseph,Rituraj Khanna,626,10.0
53,DPS,Michel Dsuza,614,10.0


#### 3.Find out students with the highest improvement for each subject from 2019-21 combining all the schools together

In [24]:
import pandas as pd

# File paths for 2019 and 2021 data
file_2019 = "Bangalore Schools 2019.xlsx"
file_2021 = "Bangalore Schools 2021.xlsx"

# Function to load data from all sheets and add school column
def load_school_data(file_path):
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names  # List of school names
    df_list = []
    for sheet in sheet_names:
        temp_df = pd.read_excel(xls, sheet_name=sheet)
        temp_df["School"] = sheet  # Assign school name from sheet name
        df_list.append(temp_df)
    return pd.concat(df_list, ignore_index=True)

# Load datasets for 2019 and 2021
df_2019 = load_school_data(file_2019)
df_2021 = load_school_data(file_2021)

# List of subjects
subjects = ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry',
            'Biology', 'History', 'Geography', 'Civics', 'Computer Science', 'Physical Education']

# Merge 2019 and 2021 datasets on Student Name and School
df_combined = df_2019.merge(df_2021, on=["Student Name", "School"], suffixes=("_2019", "_2021"))

# Calculate improvement in each subject
for subject in subjects:
    df_combined[f"{subject}_Improvement"] = df_combined[f"{subject}_2021"] - df_combined[f"{subject}_2019"]

# Find the top improving student for each subject
top_improvers = []
for subject in subjects:
    top_student = df_combined.loc[df_combined[f"{subject}_Improvement"].idxmax(), ["Student Name", "School", f"{subject}_Improvement"]]
    top_improvers.append([subject, top_student["Student Name"], top_student["School"], top_student[f"{subject}_Improvement"]])

# Convert results to DataFrame
top_improvement_df = pd.DataFrame(top_improvers, columns=["Subject", "Student Name", "School", "Improvement"])

# Display the result
print(top_improvement_df)


               Subject          Student Name        School  Improvement
0                Hindi        Sonal Tripathi  Vidya Mandir           71
1              English          Besent Kumar           DPS           59
2          Mathematics       Manyathi Shetty           DPS           67
3              Physics         Praddep Meena  Vidya Mandir           63
4            Chemistry      Manshukh Bhayani           DPS           65
5              Biology          Nitin Deewan      Birla HS           54
6              History          Rahul Bansal    St. Joseph           51
7            Geography       Subhajeet Dutta  Vidya Mandir           61
8               Civics  Sanjana Venkatramana  Vidya Mandir           65
9     Computer Science          Rashmi Desai  Vidya Mandir           82
10  Physical Education        Anamika Kumari  Vidya Mandir           58


#### 4.Identify best school for Arts, Science and Commerce streams based on marks scored by students in respective subjects for those streams in last three years

In [25]:
import pandas as pd

# File paths for 2019, 2020, and 2021 data
file_2019 = "Bangalore Schools 2019.xlsx"
file_2020 = "Bangalore Schools 2020.xlsx"
file_2021 = "Bangalore Schools 2021.xlsx"

# Function to load data from all sheets and add school column
def load_school_data(file_path):
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names  # List of school names
    df_list = []
    for sheet in sheet_names:
        temp_df = pd.read_excel(xls, sheet_name=sheet)
        temp_df["School"] = sheet  # Assign school name from sheet name
        df_list.append(temp_df)
    return pd.concat(df_list, ignore_index=True)

# Load datasets for all years
df_2019 = load_school_data(file_2019)
df_2020 = load_school_data(file_2020)
df_2021 = load_school_data(file_2021)

# List of subjects for each stream
arts_subjects = ['Hindi', 'English', 'History', 'Geography', 'Civics']
science_subjects = ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'Computer Science']
commerce_subjects = ['Hindi', 'English', 'Mathematics', 'Computer Science', 'Physical Education']

# Combine datasets for three years
df_combined = pd.concat([df_2019, df_2020, df_2021], ignore_index=True)

# Function to calculate best school for a given stream
def best_school_for_stream(subjects):
    school_avg_marks = df_combined.groupby("School")[subjects].mean().sum(axis=1)
    return school_avg_marks.idxmax()

# Identify best schools for each stream
best_school_arts = best_school_for_stream(arts_subjects)
best_school_science = best_school_for_stream(science_subjects)
best_school_commerce = best_school_for_stream(commerce_subjects)

# Display results
print("Best School for Arts:", best_school_arts)
print("Best School for Science:", best_school_science)
print("Best School for Commerce:", best_school_commerce)


Best School for Arts: Birla HS
Best School for Science: International
Best School for Commerce: Vidya Mandir


#### 5. If the marks obtained for each subject can be categorised under 5 sections like below:
    
          <=20 - Very Poor
    
    >20 - <=40 - Poor
    
    >40 - <=60 - Average
    
    >60 - <=80 - Good
    
    >80 - <=100 - Very Good
    

Then calculate for each school how many students were in each category based on the avg. marks obtained each year

In [26]:
import pandas as pd

# File paths for 2019, 2020, and 2021 data
file_2019 = "Bangalore Schools 2019.xlsx"
file_2020 = "Bangalore Schools 2020.xlsx"
file_2021 = "Bangalore Schools 2021.xlsx"

# Function to load data from all sheets and add school column
def load_school_data(file_path):
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names  # List of school names
    df_list = []
    for sheet in sheet_names:
        temp_df = pd.read_excel(xls, sheet_name=sheet)
        temp_df["School"] = sheet  # Assign school name from sheet name
        df_list.append(temp_df)
    return pd.concat(df_list, ignore_index=True)

# Load datasets for all years
df_2019 = load_school_data(file_2019)
df_2020 = load_school_data(file_2020)
df_2021 = load_school_data(file_2021)

# List of subjects for each stream
subjects = ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry', 'Biology', 'History', 'Geography', 'Civics', 'Computer Science', 'Physical Education']

# Combine datasets for three years
df_combined = pd.concat([df_2019, df_2020, df_2021], ignore_index=True)

# Function to categorize marks
def categorize_marks(marks):
    if marks <= 20:
        return "Very Poor"
    elif marks <= 40:
        return "Poor"
    elif marks <= 60:
        return "Average"
    elif marks <= 80:
        return "Good"
    else:
        return "Very Good"

# Calculate average marks per student per subject
df_combined["Average_Marks"] = df_combined[subjects].mean(axis=1)

# Categorize students based on average marks
df_combined["Category"] = df_combined["Average_Marks"].apply(categorize_marks)

# Count students in each category per school
category_counts = df_combined.groupby(["School", "Category"]).size().unstack(fill_value=0)

# Display results
print(category_counts)


Category       Average  Good
School                      
Birla HS            20    40
DPS                 41    19
International       25    35
St. Joseph          43    17
Vidya Mandir        22    38


#### 6.Which is the best school for each year 2019, 2020 and 2021 based on highest no. of students in Good and Very Good category

In [27]:
import pandas as pd

# File paths for 2019, 2020, and 2021 data
file_2019 = "Bangalore Schools 2019.xlsx"
file_2020 = "Bangalore Schools 2020.xlsx"
file_2021 = "Bangalore Schools 2021.xlsx"

# Function to load data from all sheets and add school column
def load_school_data(file_path):
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names  # List of school names
    df_list = []
    for sheet in sheet_names:
        temp_df = pd.read_excel(xls, sheet_name=sheet)
        temp_df["School"] = sheet  # Assign school name from sheet name
        df_list.append(temp_df)
    return pd.concat(df_list, ignore_index=True)

# Load datasets for all years
df_2019 = load_school_data(file_2019)
df_2020 = load_school_data(file_2020)
df_2021 = load_school_data(file_2021)

# List of subjects for each stream
subjects = ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry', 'Biology', 'History', 'Geography', 'Civics', 'Computer Science', 'Physical Education']

# Function to categorize marks
def categorize_marks(marks):
    if marks <= 20:
        return "Very Poor"
    elif marks <= 40:
        return "Poor"
    elif marks <= 60:
        return "Average"
    elif marks <= 80:
        return "Good"
    else:
        return "Very Good"

# Function to find best school per year
def best_school(df):
    df["Average_Marks"] = df[subjects].mean(axis=1)
    df["Category"] = df["Average_Marks"].apply(categorize_marks)
    category_counts = df.groupby(["School", "Category"]).size().unstack(fill_value=0)
    category_counts["Good_And_Very_Good"] = category_counts.get("Good", 0) + category_counts.get("Very Good", 0)
    best_school = category_counts["Good_And_Very_Good"].idxmax()
    return best_school

# Identify best school for each year
best_school_2019 = best_school(df_2019)
best_school_2020 = best_school(df_2020)
best_school_2021 = best_school(df_2021)

# Print results
print("Best Schools Per Year:")
print(f"2019: {best_school_2019}")
print(f"2020: {best_school_2020}")
print(f"2021: {best_school_2021}")


Best Schools Per Year:
2019: Birla HS
2020: Birla HS
2021: Vidya Mandir


#### 7. Which is the fastest-growing School in Bangalore (Overall and Streamwise)?

In [28]:
import pandas as pd

# File paths for 2019, 2020, and 2021 data
file_2019 = "Bangalore Schools 2019.xlsx"
file_2020 = "Bangalore Schools 2020.xlsx"
file_2021 = "Bangalore Schools 2021.xlsx"

# Function to load data from all sheets and add school column
def load_school_data(file_path):
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names  # List of school names
    df_list = []
    for sheet in sheet_names:
        temp_df = pd.read_excel(xls, sheet_name=sheet)
        temp_df["School"] = sheet  # Assign school name from sheet name
        df_list.append(temp_df)
    return pd.concat(df_list, ignore_index=True)

# Load datasets for all years
df_2019 = load_school_data(file_2019)
df_2020 = load_school_data(file_2020)
df_2021 = load_school_data(file_2021)

# List of subjects for each stream
subjects = ['Hindi', 'English', 'Mathematics', 'Physics', 'Chemistry', 'Biology', 'History', 'Geography', 'Civics', 'Computer Science', 'Physical Education']

# Function to categorize marks
def categorize_marks(marks):
    if marks <= 20:
        return "Very Poor"
    elif marks <= 40:
        return "Poor"
    elif marks <= 60:
        return "Average"
    elif marks <= 80:
        return "Good"
    else:
        return "Very Good"

# Function to find best school per year
def best_school(df):
    df["Average_Marks"] = df[subjects].mean(axis=1)
    df["Category"] = df["Average_Marks"].apply(categorize_marks)
    category_counts = df.groupby(["School", "Category"]).size().unstack(fill_value=0)
    category_counts["Good_And_Very_Good"] = category_counts.get("Good", 0) + category_counts.get("Very Good", 0)
    best_school = category_counts["Good_And_Very_Good"].idxmax()
    return best_school

# Identify best school for each year
best_school_2019 = best_school(df_2019)
best_school_2020 = best_school(df_2020)
best_school_2021 = best_school(df_2021)

# Calculate average marks per school for each year
school_avg_2019 = df_2019.groupby("School")[subjects].mean().mean(axis=1)
school_avg_2020 = df_2020.groupby("School")[subjects].mean().mean(axis=1)
school_avg_2021 = df_2021.groupby("School")[subjects].mean().mean(axis=1)

# Compute growth rate for each school
growth_rate = ((school_avg_2021 - school_avg_2019) / school_avg_2019) * 100
fastest_growing_school = growth_rate.idxmax()

# Identify best school streamwise
arts_subjects = ['Hindi', 'English', 'History', 'Geography', 'Civics']
science_subjects = ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'Computer Science']
commerce_subjects = ['Hindi', 'English', 'Mathematics', 'Computer Science', 'Physical Education']

arts_growth = ((df_2021.groupby("School")[arts_subjects].mean().mean(axis=1) - df_2019.groupby("School")[arts_subjects].mean().mean(axis=1)) / df_2019.groupby("School")[arts_subjects].mean().mean(axis=1)) * 100
science_growth = ((df_2021.groupby("School")[science_subjects].mean().mean(axis=1) - df_2019.groupby("School")[science_subjects].mean().mean(axis=1)) / df_2019.groupby("School")[science_subjects].mean().mean(axis=1)) * 100
commerce_growth = ((df_2021.groupby("School")[commerce_subjects].mean().mean(axis=1) - df_2019.groupby("School")[commerce_subjects].mean().mean(axis=1)) / df_2019.groupby("School")[commerce_subjects].mean().mean(axis=1)) * 100

fastest_growing_arts = arts_growth.idxmax()
fastest_growing_science = science_growth.idxmax()
fastest_growing_commerce = commerce_growth.idxmax()

# Print results
print("Best Schools Per Year:")
print(f"2019: {best_school_2019}")
print(f"2020: {best_school_2020}")
print(f"2021: {best_school_2021}")
print("\nFastest Growing Schools:")
print(f"Overall: {fastest_growing_school}")
print(f"Arts: {fastest_growing_arts}")
print(f"Science: {fastest_growing_science}")
print(f"Commerce: {fastest_growing_commerce}")


Best Schools Per Year:
2019: Birla HS
2020: Birla HS
2021: Vidya Mandir

Fastest Growing Schools:
Overall: Vidya Mandir
Arts: Vidya Mandir
Science: International
Commerce: Vidya Mandir
